<a href="https://colab.research.google.com/github/apoorvameera/fmml/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1.The accuracy of the validation set is affected by the percentage of data allocated to it in your training process. The validation set is a critical part of machine learning model development, as it helps you assess how well your model generalizes to unseen data. Here's how increasing or reducing the percentage of the validation set can impact accuracy:

1. **Increasing the Percentage of the Validation Set:**
   - **Pros:**
     - A larger validation set can provide a more reliable estimate of your model's performance. With more data in the validation set, you get a better sense of how well your model is likely to perform on new, unseen data.
     - It can help reduce overfitting, as your model has less data for training, making it more challenging for it to fit noise in the training data.

   - **Cons:**
     - If you allocate too much data to the validation set, you might not have enough data for training. This can lead to underfitting, where your model may not capture the underlying patterns in the data.

2. **Reducing the Percentage of the Validation Set:**
   - **Pros:**
     - With a smaller validation set, you have more data available for training, which can help your model learn the underlying patterns in the data better.

   - **Cons:**
     - A small validation set may not provide a robust estimate of your model's generalization performance. It can lead to overfitting, where the model performs well on the validation set but poorly on new data.

The choice of the validation set size is a trade-off. You want to strike a balance between having enough data to evaluate your model reliably and having enough data for your model to learn from during training. A common practice is to allocate around 70-80% of the data for training and the remaining 20-30% for validation. However, the specific split may vary depending on the size of your dataset and the nature of your problem.

It's also essential to use techniques like cross-validation when the dataset is limited. Cross-validation involves splitting the data into multiple subsets and training/validating the model on each subset. This helps provide a more robust estimate of model performance.

In summary, the accuracy of the validation set is influenced by the percentage of data allocated to it, and the optimal percentage depends on the specific problem and dataset. Balancing the trade-off between model evaluation and model learning is key to developing a robust machine learning model.

2.The size of the training and validation sets can have a significant impact on how well you can predict the accuracy of your model on the test set using the validation set. The validation set plays a crucial role in assessing your model's performance and making predictions about its generalization to unseen data (the test set). Here's how the sizes of these sets can affect your ability to predict accuracy on the test set:

1. **Large Training Set, Small Validation Set:**
   - In this scenario, you have a substantial amount of data for training your model, which can help it learn the underlying patterns in the data effectively.
   - However, the small validation set may not provide a reliable estimate of how well your model will generalize to the test set. This is because the validation set may not represent the diversity of data in the test set adequately.

   - If the validation set is too small, it might not be representative of the test set, and your accuracy estimate could be overly optimistic. You may miss overfitting issues that could become apparent when evaluated on a larger validation or test set.

2. **Small Training Set, Large Validation Set:**
   - With a small training set, your model may not learn the underlying patterns well, and it might underfit the data.
   - A large validation set provides a more reliable estimate of model performance on the validation data. However, if the training set is too small, your model's performance may not improve significantly.

   - While you have a better accuracy estimate on the validation set, your model may not perform well on the test set due to inadequate training.

3. **Balanced Training and Validation Set Sizes:**
   - A common practice is to allocate a significant portion of the data to the training set (e.g., 70-80%) and the remainder to the validation set.
   - This balanced approach allows your model to learn from a sufficiently large training set while providing a reasonably representative validation set.
   - It strikes a balance between model learning and evaluation, helping you obtain a more accurate estimate of the model's generalization performance on the test set.

In summary, the size of the training and validation sets is crucial for accurate predictions of model performance on the test set. A balanced approach, where you allocate a substantial portion of data to training while retaining a representative validation set, is often a good strategy. The validation set should be large enough to provide a reliable estimate of your model's performance, but not so large that it compromises the training process. This balance will help you make more accurate predictions about your model's accuracy on the test set.

3.The percentage of data to reserve for the validation set depends on the size of your dataset and the nature of your machine learning problem. There is no one-size-fits-all answer, but a common practice is to allocate around 20-30% of your data for the validation set. This percentage is often used as a starting point, but adjustments may be necessary based on the following considerations:

1. **Size of the Dataset:**
   - If you have a large dataset (thousands or millions of data points), you can afford to reserve a smaller percentage for validation. In such cases, 20% might be sufficient.

2. **Complexity of the Model:**
   - If you are training a complex model with a large number of parameters, it might be more prone to overfitting. In such cases, a larger validation set (closer to 30%) can help provide a more robust estimate of model performance.

3. **Availability of Data:**
   - If your dataset is small, you may want to allocate a larger portion to the validation set to ensure a representative sample for evaluation.

4. **Cross-Validation:**
   - If you have a very limited dataset, you can use techniques like k-fold cross-validation. This involves splitting your data into k subsets and training/validating your model on each subset. This way, you can use all your data for validation without sacrificing too much for training.

5. **Domain and Problem Specifics:**
   - Consider the characteristics of your data and the specific problem you're trying to solve. Some domains or problems may benefit from a larger or smaller validation set based on data distribution and complexity.

In practice, it's often beneficial to experiment with different validation set percentages and observe how they affect your model's performance. You can perform multiple training runs with different validation set sizes and choose the one that leads to the best overall performance and generalization.

The key is to strike a balance between having enough data for training your model effectively and having a sufficiently large validation set to estimate performance accurately. It's also important to remember that the validation set should be kept separate from the test set, and it should not be used for model tuning or hyperparameter optimization to avoid data leakage.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


1.This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Yes, averaging the validation accuracy across multiple splits in cross-validation can provide more consistent and reliable results compared to using a single validation set. Cross-validation techniques like k-fold cross-validation and leave-one-out cross-validation are designed to assess a model's performance by repeatedly splitting the data into different subsets for training and validation. Here's why averaging the validation accuracy across these multiple splits is beneficial:

1. **Reduced Variance:** Averaging the validation accuracy over multiple folds or splits helps reduce the impact of random variations in a single validation set. Different validation sets may contain different data points, and their performance estimates can vary. Averaging smooths out these variations, giving a more stable and representative measure of your model's performance.

2. **Better Generalization Assessment:** Cross-validation provides a more robust estimate of how well your model generalizes to unseen data. By assessing performance across different validation sets, you get a more comprehensive view of how the model will perform in real-world scenarios.

3. **Use of All Data for Validation:** Cross-validation ensures that every data point is used for validation in at least one fold. This maximizes the utilization of your data for performance evaluation while avoiding bias introduced by a single static validation set.

4. **Model Assessment:** Averaging the results from multiple splits helps provide a more accurate assessment of your model's performance, which is valuable when comparing different models or when tuning hyperparameters.

5. **Model Robustness:** By testing the model's performance on multiple validation sets, you can identify if the model's accuracy is consistent across different subsets of the data. This can help you assess model robustness.

In summary, averaging the validation accuracy across multiple splits in cross-validation is a good practice for obtaining more consistent and reliable performance estimates for your machine learning model. It helps mitigate the variability associated with a single validation set and provides a more comprehensive evaluation of your model's generalization capabilities.

2.Cross-validation, which involves averaging the validation accuracy across multiple splits of your data, provides a more accurate estimate of how well your model is likely to perform on unseen data (test accuracy). However, it's important to clarify the distinction between validation accuracy and test accuracy:

1. **Validation Accuracy:** Validation accuracy is an estimate of how well your model performs on a dataset that it hasn't seen during training. This dataset is part of your original data but is reserved for model evaluation and hyperparameter tuning. Validation accuracy is used to make decisions about model selection and configuration.

2. **Test Accuracy:** Test accuracy is a measure of how well your model generalizes to completely new, unseen data. It's typically assessed using a separate dataset that the model has never encountered before, and it serves as the final evaluation of your model's performance.

Here's how cross-validation contributes to obtaining a more accurate estimate of test accuracy:

- Cross-validation provides a more robust and reliable estimate of how well your model is likely to generalize to new data. By training and validating the model on multiple subsets of your dataset, it helps account for data variability and reduces the risk of overfitting to a single validation set.

- While cross-validation can provide a more accurate estimate of how well your model will perform on the test data, it's important to note that it's still an estimate. The actual test accuracy on entirely new, unseen data may vary, but cross-validation is a strong indicator of your model's generalization capabilities.

- Cross-validation is a valuable tool for model selection and hyperparameter tuning. When you've chosen a model and its configuration based on cross-validation results, you can then assess its performance on the test data to get a final, realistic measure of how well it will perform in a real-world setting.

In summary, while cross-validation doesn't directly give you the exact test accuracy, it provides a more accurate estimate of your model's generalization performance compared to using a single validation set. It's a crucial step in the model development process and helps you make more informed decisions about your model's likely performance on unseen data. The test accuracy, obtained using a completely independent dataset, is the final measure of your model's real-world performance.

3.The number of iterations, or folds, in cross-validation can impact the quality of your estimate. Cross-validation, such as k-fold cross-validation, involves splitting your dataset into multiple subsets, training and validating your model multiple times. Here's how the number of iterations affects the estimate:

1. **More Iterations (Higher k):**
   - Increasing the number of iterations (higher k) in cross-validation can provide a more reliable estimate of your model's performance. With more folds, you obtain a more comprehensive assessment of how well your model generalizes to different data subsets.

   - A larger k reduces the impact of random variations in the validation sets, resulting in a more stable and representative performance estimate.

   - You also make more efficient use of your data for validation, as each data point is used for validation in multiple folds. This maximizes the data's contribution to the performance estimate.

2. **Fewer Iterations (Lower k):**
   - Using a smaller number of iterations in cross-validation can still provide a reasonable estimate of model performance, but it may be more susceptible to variations in the validation set composition.

   - With a smaller k, each fold represents a larger portion of the data, and the estimate may be influenced more by the specific composition of the validation sets in those folds.

In general, using a higher number of iterations (higher k) in cross-validation tends to provide a better estimate of your model's performance, as it offers a more robust and stable assessment of generalization. However, there are trade-offs to consider:

- As k increases, the computational cost of cross-validation also goes up, as you train and validate the model multiple times. This can be a concern if you have limited computational resources.

- In cases where you have a small dataset, very high k values may result in each fold having extremely limited data for training and validation, which can lead to less accurate estimates.

- A common choice for k in k-fold cross-validation is often 5 or 10, which strikes a balance between computational cost and accuracy of the estimate. However, the choice of k can vary based on your specific dataset and computational resources.

In summary, increasing the number of iterations (higher k) in cross-validation generally leads to a better estimate of your model's performance, but the choice of k should consider trade-offs between accuracy, computational resources, and the dataset size. It's essential to choose a value of k that aligns with the specific needs and constraints of your project.

4.Increasing the number of iterations in cross-validation can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it doesn't entirely replace the need for a sufficiently sized dataset for training and validation. Here's how increasing iterations affects dealing with small datasets:

1. **Dealing with a Very Small Training Dataset:**
   - If your training dataset is very small, increasing the number of iterations can be helpful because it allows you to utilize the available training data more effectively.
   - With more iterations, the model will be trained on different subsets of the small training data, which can help it capture a broader range of patterns.
   - However, it's important to note that having a very small training dataset remains a limitation. Cross-validation can only extract the maximum value from the available data but can't generate more data.

2. **Dealing with a Very Small Validation Dataset:**
   - If your validation dataset is very small, increasing the number of iterations can also be beneficial. More iterations reduce the risk of overfitting to the specific validation data in each fold, providing a more robust estimate of model performance.
   - With a small validation dataset, it's more likely to experience variability in the performance estimates, which can be smoothed out with more iterations.

While increasing the number of iterations can help in both cases, it's essential to consider the limitations:

- If your training dataset is extremely small, the model may struggle to generalize well, even with cross-validation. The limited data may result in high variability in the model's performance.

- If your validation dataset is tiny, it can still limit the precision of the performance estimates, especially for model selection or hyperparameter tuning.

In situations with very small datasets, it's crucial to acknowledge these limitations and explore other techniques to supplement your data, such as data augmentation, transfer learning, or domain-specific knowledge. Additionally, make the most of the data you have by using techniques like cross-validation to extract the best possible performance estimates.